# `pyGIMLi(emg3d)` - Inversion

**Using `pyGIMLi` as an inversion framework for `emg3d` computations.**

=> Create an environment using the provided `environment.yml`: `mamba env create`.

**Current Limitations**
- Only isotropic models supported

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm

import emg3d
import pygimli as pg

# Requires currently branch inversion:
# pip install git+https://github.com/emsig/emg3d@inversion
from emg3d.inversion.pygimli import Inversion

## Load survey (incl. data), initial model, and create a Simulation

In [ ]:
inp_data = emg3d.load('pginv-7x7.h5')        # Finer model (but still coarse)

inp_survey = inp_data['survey']
inp_model = inp_data['model']
inp_grid = inp_model.grid
inp_model

In [ ]:
# The model is resistivity. Change it to conductivity
# TODO: make this internally happen, so that pyGIMLi
# always gets a conductivity model!
con_model = emg3d.Model(inp_grid, 1/inp_model.property_x, mapping='Conductivity')
con_model

In [ ]:
# For dev-purposes, we only select one source
# [for dev-purposes it is also a very simple model]
inp_survey = inp_survey.select(sources=['TxED-1', 'TxED-7'], frequencies='f-1')
#inp_survey = inp_survey.select(sources=['TxED-1', 'TxED-7'], frequencies=['f-1', 'f-5'])
#inp_survey = inp_survey.select(frequencies='f-1')
#inp_survey = inp_survey.select(sources=['TxED-1', 'TxED-3', 'TxED-5', 'TxED-7'], frequencies=['f-1', 'f-3', 'f-5'])
inp_survey

## Create a Starting Model

In [ ]:
hx = np.ones(58)*250.0
hy = np.ones(26)*250.0
hz = np.ones(15)*250.0
grid = emg3d.TensorMesh([hx, hy, hz], [-7250, -3250, -3500])

model = emg3d.Model(grid, 1.0, mapping='Conductivity')
model.property_x[:, :, -1:] = 3.33

# QC
grid

## Create a Simulation

In [ ]:
# Create an emg3d Simulation instance
sim = emg3d.simulations.Simulation(
    survey=inp_survey,
    model=model,
    gridding='both', #'same',  # I would like to make that more flexible in the future
    gridding_opts={'vector': 'xyz'},
    max_workers=15,    # Adjust as needed
    receiver_interpolation='linear',  # Currently necessary for the gradient
    ### Tolerance: TODO: different ones for forward and adjoints
    solver_opts={'tol': 1e-3},                # Just for dev-purpose
    tqdm_opts=False,  # Switch off verbose progress bars
    #verb=3,
)
sim

## Instantiate and run inversion

In [ ]:
INV = Inversion(fop=sim) #, verbose=True, debug=True)
INV.inv.setCGLSTolerance(10)  # is _absolute_, not _relative_
INV.inv.setMaxCGLSIter(30)

#INV.dataTrans = pg.trans.TransSymLog(sim.survey.noise_floor)

# Create region markers
# Set marker -> water is 1, subsurface is 0
INV.inv_mesh.setCellMarkers(pg.z(INV.inv_mesh.cellCenters()) > 0)
# pg.show(INV.inv_mesh)

# , zWeight=0.2
# , correlationLengths=[1000, 1000, 300])

INV.setRegularization(0, limits=(0.001, 2), zWeight=0.2, startModel=1)
INV.setRegularization(1, limits=(3.32, 3.34), startModel=3.33) # , single=True)
# 0 only damping (minimum length; only useful with good starting model and isReference=True in INV)
# 1 1st deriv - smoothing
# 2 2nd deriv
# 10, 20 - mixed form


# außerdem kann man der Inversion inv.setPostStep(fct) und inv.setPreStep(fct) mitgeben,
# die als def fct(iter, inv): die Iterationszahl und die Inversionsinstanz bekommt,
# damit kann man machen was man will 

In [ ]:
errmodel = INV.run(
    maxIter=5, # just to test
    lam=1,  # btw 1-100
    #verbose=True,
    #startModel=...,
    #isReference=True,
)

## Plots

In [ ]:
# Add inversion result to data;
# I should wrap these few lines into a function, as they are used above as well
idata = np.ones(sim.survey.shape, dtype=sim.data.observed.dtype)*np.nan
x = np.asarray(INV.response)
idata[sim.survey.isfinite] = x[:x.size//2] + 1j*x[x.size//2:]
sim.survey.data['inv'] = sim.survey.data.observed.copy(data=idata)

# Compute the 2-layer model as comparison
sim.clean('computed')
sim.model = model
sim.compute()  # <- puts it into 'synthetic'

In [ ]:
# Store final model as model to be saved with the simulation
sim.model = emg3d.Model(grid, np.array(INV.model), mapping='Conductivity')
sim.to_file('Simulation-7x7-Test.h5')

In [ ]:
popts1 = {'cmap':'Spectral_r','norm':LogNorm(vmin=0.3, vmax=100)}
popts2 = {'edgecolors': 'grey', 'linewidth': 0.5, 'cmap':'Spectral_r','norm':LogNorm(vmin=0.3, vmax=100)}
opts = {'v_type': 'CC', 'normal': 'Y'}

rec_coords = sim.survey.receiver_coordinates()
src_coords = sim.survey.source_coordinates()

fig, axs = plt.subplots(2, 3, figsize=(9, 6), constrained_layout=True)#, sharex=True, sharey='col')
(ax1, ax2, ax3), (ax4, ax5, ax6) = axs

# True model
out1, = inp_grid.plot_slice(inp_data['true_model'].property_x.ravel('F'), ax=ax1, pcolor_opts=popts1, **opts)
ax1.set_title(f"True Model (Ohm.m)")
ax1.plot(rec_coords[0], rec_coords[1], 'bv')
ax1.plot(src_coords[0], src_coords[1], 'r*')

# Start model
out2, = model.grid.plot_slice(1/model.property_x.ravel('F'), ax=ax2, pcolor_opts=popts1, **opts)
ax2.set_title(f"Start Model (Ohm.m)")

# Final inversion model
out3, = model.grid.plot_slice(1/sim.model.property_x.ravel('F'), ax=ax3, pcolor_opts=popts1, **opts)
ax3.set_title(f"Final Model (Ohm.m)")

opts['normal'] = 'Z'
opts['ind'] = 4

# True model
out4, = inp_grid.plot_slice(inp_data['true_model'].property_x.ravel('F'), ax=ax4, pcolor_opts=popts1, **opts)
ax4.set_title(f"True Model (Ohm.m)")
ax4.plot(rec_coords[0], rec_coords[2], 'bv')
ax4.plot(src_coords[0], src_coords[2], 'r*')

# Start model
out5, = model.grid.plot_slice(1/model.property_x.ravel('F'), ax=ax5, pcolor_opts=popts1, **opts)
ax5.set_title(f"Start Model (Ohm.m)")

# Final inversion model
out6, = model.grid.plot_slice(1/sim.model.property_x.ravel('F'), ax=ax6, pcolor_opts=popts1, **opts)
ax6.set_title(f"Final Model (Ohm.m)")

for ax in axs.ravel():
    ax.axis('equal')

plt.colorbar(out1, ax=axs, orientation='horizontal', fraction=.1, shrink=.8, aspect=30)
plt.savefig('ModelFit-7x7-Test.png', dpi=300)

In [ ]:
def pos(data):
    """Return positive data; set negative data to NaN."""
    return np.where(data > 0, data, np.nan)
    

height = sim.survey.shape[2]*1.7+2
fig, axs = plt.subplots(sim.survey.shape[2], 3, figsize=(9, height), constrained_layout=True, sharex=True, sharey=True)
axs = np.atleast_2d(axs)  # So it works if only 1 freq

# Data
obs = sim.data.observed
syn = sim.data.inv
syn.data[~np.isfinite(obs.data)] = np.nan+1j*np.nan
two = sim.data.synthetic
two.data[~np.isfinite(obs.data)] = np.nan+1j*np.nan

rec_coords = sim.survey.receiver_coordinates()

for ii, freq in enumerate(sim.survey.frequencies.keys()):
    for i, src in enumerate(sim.survey.sources.keys()):
        #if i in [1, 3, 5]:
        #    continue
        
        # Observed data
        axs[ii, 0].plot(rec_coords[0], abs(obs.loc[src, :, freq].data), f"C{i}.",
                        label='observed' if i == 0 else '')
        axs[ii, 1].plot(rec_coords[0], pos(obs.loc[src, :, freq].data.real), f"C{i}.",
                        label='positive' if i == 0 else '')
        axs[ii, 1].plot(rec_coords[0], pos(-obs.loc[src, :, freq].data.real), f"C{i}x", ms=3,
                        label='negative' if i == 0 else '')
        axs[ii, 2].plot(rec_coords[0], pos(obs.loc[src, :, freq].data.imag), f"C{i}.")
        axs[ii, 2].plot(rec_coords[0], pos(-obs.loc[src, :, freq].data.imag), f"C{i}x", ms=3)
        
        # Start model
        axs[ii, 0].plot(rec_coords[0], abs(two.loc[src, :, freq].data), f"C{i}--", alpha=.4, lw=0.5,
                        label='start' if i == 0 else '')
        axs[ii, 1].plot(rec_coords[0], abs(two.loc[src, :, freq].data.real), f"C{i}--", alpha=.4, lw=0.5)
        axs[ii, 2].plot(rec_coords[0], abs(two.loc[src, :, freq].data.imag), f"C{i}--", alpha=.4, lw=0.5)
        
        # Inverted data
        axs[ii, 0].plot(rec_coords[0], abs(syn.loc[src, :, freq].data), f"k-", lw=0.8,
                        label='inversion' if i == 0 else '')
        axs[ii, 1].plot(rec_coords[0], pos(syn.loc[src, :, freq].data.real), f"k-", lw=0.8)
        axs[ii, 1].plot(rec_coords[0], pos(-syn.loc[src, :, freq].data.real), f"k--", lw=0.8)
        axs[ii, 2].plot(rec_coords[0], pos(syn.loc[src, :, freq].data.imag), f"k-", lw=0.8,
                        label='positive' if i == 0 else '')
        axs[ii, 2].plot(rec_coords[0], pos(-syn.loc[src, :, freq].data.imag), f"k--", lw=0.8,
                        label='negative' if i == 0 else '')
        

    axs[ii, 0].set_ylabel(f'$f={sim.survey.frequencies[freq]}$ Hz')     

        
axs[0, 0].set_yscale('log')
axs[0, 0].set_ylim([3e-16, 2e-7])

axs[0, 0].set_title('Amplitude: $|E_x|$ (V/m)')
axs[0, 1].set_title('Real $|\Re(E_x)|$ (V/m)')
axs[0, 2].set_title('Imaginary $|\Im(E_x)|$ (V/m)')

axs[0, 0].legend(framealpha=1)
axs[0, 1].legend(framealpha=1)
axs[0, 2].legend(framealpha=1)

axs[-1, 1].set_xlabel('x-coordinate (m)')
plt.savefig('ResponseFit-7x7-Test.png', dpi=300)
plt.show()

In [ ]:
emg3d.Report()